# Data Wrangling in R (part 2)

## Table of Contents

- [Understanding Data](#data)
- [Data Frames](#dataframe)
- [Working with CSV Data](#csv)
- [Factors](#fac)
- [Manipulating Data with dplyr](#dplyr)
- [The Pipe Operator](#pipe)
- [Tibble](#tib)
- [Joining Data Frames Together](#join)